# Segmentation and motion correction of in-vivo recordings

Before running this notebook, generate an ROI called `correctionReference.npy` using [Robopy](https://github.com/fedeceri85/robopy2). This ROI should be placed in the same folder as the recording and contain a trace where the "jumps" are visible as decreases (ideally) or increases in fluorescence.

This notebook contains the preprocessing steps and segmentation routines for in vivo recordings. Some utilities used in this notebook are:

- `thorlabsFile` from `movieTools`: This class contains the actual recording data, functions to smooth and filter it, and display it in a Napari viewer. The `thorlabsFile` object is passed to other routines to extract data. When loading files, they are automatically smoothed with a 3D Gaussian kernel (2x2x2).
- `jumpFramesFinder` from `visualisationTools`: This displays an ipywidget interface with tools to remove out-of-focus frames. This interface displays the `correctionReference` generated using Robopy (first step of the analysis).
- `jupyterpy` from `movieTools`: This interface connects to the currently displayed `thorlabsFile` and contains buttons for the segmentation of hair cells, calcium waves, and fibers.
- `maskMatching` from `movieTools`: This is an ipywidgets/Napari interface that combines repetitive recordings of the same field of view to generate a `matchingMask`, i.e., a label image where hair cell ROIs are color-coded with the same color.


In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%pylab
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import os
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import argrelextrema,argrelmin,argrelmax
import sys
sys.path.append('../../src')

import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
import visualisationTools as vu
import traceUtilities as tu
import ast 
from movieTools import thorlabsFile

parameterFolder = '../../parameters/'
fileHeader = 'Pax2_Calciumwaves'

inputFilename = os.path.join('../../',fileHeader)+'.xlsx'
corrFilename = os.path.join(parameterFolder,fileHeader)+ '_with_corr_params.csv'
jumpFrameFilename = os.path.join(parameterFolder,fileHeader+'jump_frames.csv')
jumpFrameMaxFilename = os.path.join(parameterFolder,fileHeader+'jumpMax_frames.csv')
correctionReferenceTraceFile = os.path.join(parameterFolder,fileHeader+'correctionReference.csv')

In [3]:
alldata = pd.read_excel(inputFilename)
alldata = alldata[alldata['discard']!=1]

try:
    corr_param = pd.read_csv(corrFilename)
except:
    print('No jump correction parameter file')

try:
    allminima = pd.read_csv(jumpFrameFilename)
except FileNotFoundError:
    allminima = pd.DataFrame()

try:
    allmaxima = pd.read_csv(jumpFrameMaxFilename)
except FileNotFoundError:
    allmaxima = pd.DataFrame()


try:
    correctionReferenceTrace = pd.read_csv(correctionReferenceTraceFile)
except FileNotFoundError:
    correctionReferenceTrace = pd.DataFrame()

In [4]:
for el in alldata['Folder'].unique():
    try:
        alldata.loc[alldata['Folder']==el,'Window left'] = corr_param.loc[corr_param['Folder']==el,'Window left'].values[0]
        alldata.loc[alldata['Folder']==el,'Window right'] = corr_param.loc[corr_param['Folder']==el,'Window right'].values[0]
        alldata.loc[alldata['Folder']==el,'Minima order'] = corr_param.loc[corr_param['Folder']==el,'Minima order'].values[0]



    except (IndexError, NameError, KeyError):
        print('Could not find correction parameters for '+ el+' Setting to default.')
        alldata.loc[alldata['Folder']==el,'Window left'] = 0
        alldata.loc[alldata['Folder']==el,'Window right'] = 0
        alldata.loc[alldata['Folder']==el,'Minima order'] = 50 



    try:
        alldata.loc[alldata['Folder']==el,'Window Max left'] = corr_param.loc[corr_param['Folder']==el,'Window Max left'].values[0]
        alldata.loc[alldata['Folder']==el,'Window Max right'] = corr_param.loc[corr_param['Folder']==el,'Window Max right'].values[0]
        alldata.loc[alldata['Folder']==el,'Maxima order'] = corr_param.loc[corr_param['Folder']==el,'Maxima order'].values[0]
    except (IndexError, NameError, KeyError):
        alldata.loc[alldata['Folder']==el,'Window Max left'] = 0
        alldata.loc[alldata['Folder']==el,'Window Max right'] = 0
        alldata.loc[alldata['Folder']==el,'Maxima order'] = 50

alldata.loc[alldata['Window left'].isna(),'Window left'] = 0
alldata.loc[alldata['Window right'].isna(),'Window right'] = 0
alldata.loc[alldata['Minima order'].isna(),'Minima order'] = 50

alldata.loc[alldata['Folder'].isna(),'Window Max left'] = 0
alldata.loc[alldata['Folder'].isna(),'Window Max right'] = 0
alldata.loc[alldata['Folder'].isna(),'Maxima order'] = 50

alldata['Window left'] = alldata['Window left'].astype(int)
alldata['Window right'] = alldata['Window right'].astype(int)
alldata['Minima order'] = alldata['Minima order'].astype(int)

alldata['Window Max left'] = alldata['Window Max left'].astype(int)
alldata['Window Max right'] = alldata['Window Max right'].astype(int)
alldata['Maxima order'] = alldata['Maxima order'].astype(int)


alldata.loc[alldata['Minima order']==1,'Minima order'] = 50
alldata.loc[alldata['Maxima order']==1,'Maxima order'] = 50

try:
    for el in alldata['Folder'].unique():
        alldata.loc[alldata['Folder']==el,'ExtraCorrectionIntervals'] = corr_param.loc[corr_param['Folder']==el,'ExtraCorrectionIntervals'].values[0]
except:
    pass

for j,el in alldata.iterrows():
    try:
        alldata.at[j,'ExtraCorrectionIntervals'] = ast.literal_eval(alldata.at[j,'ExtraCorrectionIntervals'] )
    except (ValueError,KeyError):
         alldata.at[j,'ExtraCorrectionIntervals'] = np.nan
alldata['ExtraCorrectionIntervals'] =  alldata['ExtraCorrectionIntervals'].astype('object')


try:
    for el in alldata['Folder'].unique():
        alldata.loc[alldata['Folder']==el,'TemplateIntervals'] = corr_param.loc[corr_param['Folder']==el,'TemplateIntervals'].values[0]
except:
    pass

for j,el in alldata.iterrows():
    try:
        alldata.at[j,'TemplateIntervals'] = ast.literal_eval(alldata.at[j,'TemplateIntervals'] )
    except (ValueError,KeyError):
         alldata.at[j,'TemplateIntervals'] = np.nan

alldata['TemplateIntervals'] =  alldata['TemplateIntervals'].astype('object')

In [5]:
master = alldata.copy().reset_index()


# 1- Jump Correction. 
Remember to run the block after this every now and then to save the results in case this notebook crashes
`jumpFramesFinder` is a comprehensive tool that combines interactive widgets, data processing, and visualization to assist users in identifying and correcting frame jumps in their datasets.
The jumpFramesFinder function is designed to facilitate the identification and correction of frame jumps in a in vivo movie. This function utilizes a variety of widgets from the `ipywidgets` library to create an interactive user interface  that allows users to adjust parameters and visualize the effects of these adjustments in real-time.

The user can utilise various sliders to control various parameters such as the trace number, window sizes for minima and maxima detection, and the order of minima and maxima. These sliders allow users to fine-tune the detection of frame jumps by adjusting the sensitivity and range of the detection algorithm.

Users can perform specific actions, such as loading original or corrected movie, saving processed files, and creating templates for further analysis.

The function also sets up a plot using `plotly.graph_objects` to visualize the original and corrected traces, as well as the detected jumps. This visualization helps users to see the impact of their adjustments and make decisions about the parameters they set.

The corrected movie is visualised through the `thorlabsFile` object in a napari window.

After this step, launch the `batchModtionCorrect.ipynb` notebook to motion correct the "jumpCorrected.tif" files, which will be saved as `jumpCorrected-mc.tif` files. After that, come back to this notebook, and run this block to load the motion corrected files, to be used for segmentation.

In [ ]:
tb = thorlabsFile()
vu.jumpFramesFinder(master,allminima,allmaxima,correctionReferenceTrace,tb)

## Save parameters block. Uncomment it and run it while doing the previous step to prevent data loss.

In [ ]:
#master.to_csv(corrFilename)
#allminima.to_csv(jumpFrameFilename)
#allmaxima.to_csv(jumpFrameMaxFilename)
#correctionReferenceTrace.to_csv(correctionReferenceTraceFile)


## The next block displays the jupyterpy interface. This connects to the opened image in napari (the current thorlabsFile object, it should be loaded using the jumpFramesFinder above) and provides buttons with specific analysis steps for each type of experiment.  

The jupyterPy function is designed to create an interactive user interface (UI) for analyzing image data within a Jupyter notebook. This function utilizes the ipywidgets library to create various interactive widgets, such as buttons, sliders, and text inputs, which are then organized into a tabbed layout for different analysis tasks.
The istance of jupyterPy connects to the movie displayed in the existing napari windows (passed through the thorlabsFile object.)

The software allows to perform specific segmentation analysis for IHCs, Calcium waves, and afferent terminals. 
For IHCs, the software uses cellpose to automatically identify the cell bodies. The segmentation is shown as a `label` layer (named 'Masks') in Napari. Users can modify the existing ROIs using Napari tools (e.g, splitting ROIs). The function also allows to plot and save the fluorescence profile of the individual ROIs. 


Instructions for Ca wave detection:


1 - Start from the raw image. Select a background frmae where the fluorescence is quite low (now always at the beginning). Subtract it

2 - Subtract an integer number (Subtract X Background) to reduce noise. Usually 20 is good.

3 - If desired, bin 2 the image (a quarter of the frame size) to save time. 

4 - If desired, downsample time (one frame every 2)

5 - Select spot and outline sigma for the voronoi.

6 - execute the voronoi.

7 - inspect the labels, delete single supoorting cell events and merge/split labels using Plugins->Napari segment blobs and things...->manually merge/split labels. 

8 - If labels are deleted or merged, rearrange the labels so they are numbered sequentially.

9 - If the recording has been scaled at the beginning, scale back the labels to original size before saving!

10 - Save

In [ ]:
from movieTools import jupyterPy
JP = jupyterPy(tb)

# Generate kymographs for all recordings

In [ ]:
import cupy as cp
from cupyx.scipy import ndimage
import tifffile 
def calculateKymo(data, hcline):
    """
    Calculate kymographs from the given data along a specified line.
    
    Parameters:
    data (numpy.ndarray): 3D array of shape (frames, height, width) containing the image data.
    hcline (numpy.ndarray): 2D array of shape (n_points, 2) containing the coordinates of a line drawn on the IHCs.
    
    Returns:
    numpy.ndarray: 2D array containing the kymographs.
    """
    MAXCHUNKSIZE = 10000  # Maximum chunk size for processing
    Nchunks = data.shape[0] // MAXCHUNKSIZE  # Number of chunks

    kymos = []
    # Calculate the coordinates along the line
    x = []
    y = []
    for i in range(hcline.shape[0] - 1):
        x0 = np.arange(hcline[i, 0], hcline[i + 1, 0])
        x.append(x0)
        y.append(np.linspace(hcline[i, 1], hcline[i + 1, 1], x0.size))
    x = np.hstack(x)
    y = np.hstack(y)
    cphcline = cp.array(np.vstack([y, x]))  # Convert to CuPy array

    for chunk in range(Nchunks):
        cpdata = cp.array(data[chunk * MAXCHUNKSIZE:(chunk + 1) * MAXCHUNKSIZE, :, :])  # Load chunk into GPU
        
        # Loop through the frames
        kymo = []
        for i in range(cpdata.shape[0]):
            img1 = ndimage.map_coordinates(cpdata[i, :, :], cphcline)
            # Average over neighboring pixels
            for j in [-2, -1, 1, 2]:
                img1 += ndimage.map_coordinates(cpdata[i, :, :], cphcline + cp.array([[j], [0]]))
            kymo.append(img1 / 5.0)
        kymos.append(np.vstack(kymo).get())  # Move result back to CPU

        del cpdata
        cp.get_default_memory_pool().free_all_blocks()  # Free GPU memory
    
    # Process remaining frames if any
    if data.shape[0] % MAXCHUNKSIZE != 0:
        cpdata = cp.array(data[Nchunks * MAXCHUNKSIZE:, :, :])
        
        # Loop through the frames
        kymo = []
        for i in range(cpdata.shape[0]):
            img1 = ndimage.map_coordinates(cpdata[i, :, :], cphcline)
            # Average over neighboring pixels
            for j in [-2, -1, 1, 2]:
                img1 += ndimage.map_coordinates(cpdata[i, :, :], cphcline + cp.array([[j], [0]]))
            kymo.append(img1 / 5.0)
        kymos.append(np.vstack(kymo).get())  # Move result back to CPU

        del cpdata
        cp.get_default_memory_pool().free_all_blocks()  # Free GPU memory

    kymos = np.vstack(kymos)  # Combine all chunks
    return kymos

In [ ]:
# Calculate kymographs for all the data
drive = 'D'
movementum = 20#um how much to shift the line in y to intersect the bulk of the ger instead of the ihcs
for i,el in master.iterrows():
    #if i ==44:
        folder = drive+el['Folder'][1:]
        print(folder,i)
        hcline = pd.read_csv(os.path.join(folder,'hcs.csv'))
        umpx = el['um/pixel']
        movementY = movementum / umpx

        hcline_shifted = hcline - [0,movementY] 

        data = tifffile.imread(os.path.join(folder,'processedMovies','1-jumpCorrected-mc.tif'))

        kymoHC = calculateKymo(data,hcline.values)
        kymoSCs = calculateKymo(data,hcline_shifted.values)
        
        img = np.stack((kymoHC,kymoSCs,zeros(kymoHC.shape)),axis=0)
        tifffile.imwrite(os.path.join(folder,'processedMovies','kymoHCs.tif'),img,metadata = {'axes':'YXC','PhysicalSizeX': umpx, 'PhysicalSizeXUnit': 'µm',
                                                                'PhysicalSizeY': 1.0/el['fps'],'PhysicalSizeYUnit': 's'},photometric='rgb')

        
        

# Calculate extension of hair cell vs scs calcium waves

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import shapely

In [ ]:
masterIHCvsSC = pd.DataFrame()

# Iterate through each row in the master dataframe
for i, el in master.iterrows():
    # Check if the kymoShapes.csv file exists in the processedMovies folder
    if os.path.exists(os.path.join(el['Folder'], 'processedMovies', 'kymoShapes.csv')):
        df = pd.read_csv(os.path.join(el['Folder'], 'processedMovies', 'kymoShapes.csv'))
        
        # Ensure there is an even number of waves (last index must be odd)
        if df['index'].unique().max() % 2 != 1:
            print('Error, waves not matched')

        # Iterate through each unique index in the dataframe
        for index in df['index'].unique():
            this_masterIHCSSCS = pd.Series(dtype=float64)
            this_df = df[df['index'] == index]
            
            # Determine the event type based on the shape type
            if this_df['shape-type'].iloc[0] == 'rectangle':
                this_masterIHCSSCS['Event type'] = 'IHCs'
            elif this_df['shape-type'].iloc[0] == 'ellipse':
                this_masterIHCSSCS['Event type'] = 'GER'
            
            # Calculate event extension in micrometers
            this_masterIHCSSCS['Event extension um'] = (this_df['axis-1'].max() - this_df['axis-1'].min()) * el['um/pixel']
            this_masterIHCSSCS['Time (frame)'] = this_df['axis-0'].min()
            this_masterIHCSSCS['Time (s)'] = this_df['axis-0'].min() * el['fps']
            this_masterIHCSSCS['Folder'] = el['Folder']
            this_masterIHCSSCS['Mouse ID'] = el['Mouse ID']
            this_masterIHCSSCS['Age'] = el['Age']
            
            # Create a polygon from the axis-1 and axis-0 coordinates
            this_masterIHCSSCS['Polygon'] = shapely.geometry.Polygon(list(zip(this_df['axis-1'], this_df['axis-0'])))

            # Append the series to the masterIHCvsSC dataframe
            masterIHCvsSC = masterIHCvsSC.append(this_masterIHCSSCS, ignore_index=True)

# Parse the dataframe two by two, check that one event is IHCs and one is GER, and check that the two events intersect in the kymograph
for j in range(masterIHCvsSC.shape[0] // 2):
    pol1 = masterIHCvsSC.iloc[j * 2]['Polygon']
    pol2 = masterIHCvsSC.iloc[j * 2 + 1]['Polygon']
    type1 = masterIHCvsSC.iloc[j * 2]['Event type']
    type2 = masterIHCvsSC.iloc[j * 2 + 1]['Event type']

    index = masterIHCvsSC.index[j * 2]
    index2 = masterIHCvsSC.index[j * 2 + 1]
    
    # Check if the polygons intersect
    if not pol1.intersects(pol2):
        print('ERROR ! not intersecting consecutive shapes')
    # Check if the two consecutive events are of the same type
    elif type1 == type2:
        print('ERROR! two events of the same type consecutive')
    else:
        # Assign a unique event number to the intersecting events
        masterIHCvsSC.loc[index, 'Unique event number'] = j
        masterIHCvsSC.loc[index2, 'Unique event number'] = j


In [ ]:
masterIHCvsSC.to_csv('..\\..\\Data for figures\\CalciumWaves\\masterIHCvsSC_kymographquantification.csv')

# N numbers

In [ ]:
master.groupby(['Age','Mouse ID']).count()['Folder']